# 🚀 AIMO 3: The Complete Inference Server Starter

> **Status**: ✅ Verified & Working | **Pattern**: Gateway ↔ Inference Server

Welcome to the **AI Mathematical Olympiad - Progress Prize 3**! This competition uses a unique **Inference Server** pattern which can be tricky to set up. This notebook provides a **robust, verified template** that handles all the complex connection logic so you can focus 100% on your math solver.

### 🌟 Features
- **Plug-and-Play Solver**: Just replace the `solve_math_problem` function with your LLM/Agent logic.
- **Correct Signatures**: Handles the tricky `predict(*args)` unpacking from the Gateway.
- **Robust Error Handling**: includes local verification to ensure `submission.parquet` is generated correctly.
- **Type Safety**: Explicitly handles Polars types to prevent submission errors.

---

## 🛠️ The Pipeline Explained

Unlike standard competitions, we don't just generate a CSV. We run a gRPC server that the Competition Gateway talks to.

```mermaid
flowchart LR
    style Gateway fill:#f9f,stroke:#333,stroke-width:2px
    style Server fill:#bbf,stroke:#333,stroke-width:2px
    
    subgraph Kaggle_Environment [Kaggle Environment]
        Gateway[Competition Gateway] <==> Server[Your Inference Server]
    end
    
    Gateway -- "1. Send Problem (ID, Text) " --> Server
    Server -- "2. Solve & Return Answer" --> Gateway
    Gateway -- "3. Write to submission.parquet" --> Output[(submission.parquet)]
```

## 1. Setup & Configuration

In [ ]:
import os
import re
import sys
import warnings
import polars as pl

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Determine environment
IS_KAGGLE = os.path.exists('/kaggle/input')
IS_RERUN = os.getenv('KAGGLE_IS_COMPETITION_RERUN') is not None

# Add competition library to path
if IS_KAGGLE:
    sys.path.insert(0, '/kaggle/input/ai-mathematical-olympiad-progress-prize-3')
else:
    sys.path.insert(0, os.getcwd())

print(f"🔹 Environment: {'Kaggle' if IS_KAGGLE else 'Local'}")
print(f"🔹 Mode: {'Competition Rerunning (Server Active)' if IS_RERUN else 'Interactive/Testing'}")

## 2. 🧠 Your Math Solver Logic

> **👉 EDIT HERE**: This is the only part you need to change! Plug in your LLM, Agents, or custom logic below.

In [ ]:
from sympy import symbols, solve, simplify, sympify

# --- ⬇️ REPLACE THIS WITH YOUR LLM LOGIC ⬇️ ---

def clean_latex(s):
    """Basic cleanup for LaTeX strings."""
    s = s.replace('$', '').replace('\\times', '*').replace('\\cdot', '*')
    s = s.replace('\\div', '/').replace('\\frac', '').replace('\\', '')
    return s

def solve_math_problem(problem: str) -> int:
    """
    Solves a single math problem and returns an integer answer.
    output: int between 0 and 999999
    """
    problem = str(problem).strip()
    
    # 1. Try Simple Equation Solver (for '4+x=4' type problems)
    if 'solve' in problem.lower() and '=' in problem:
        try:
            # Extract equation parts
            m = re.search(r'\$?([^$=]+)=([^$]+)\$?', problem)
            if m:
                lhs, rhs = clean_latex(m.group(1)), clean_latex(m.group(2).split('for')[0].strip())
                x = symbols('x')
                # Solve for x
                sols = solve(sympify(lhs) - sympify(rhs), x)
                if sols:
                    return int(sols[0]) % 100000
        except:
            pass
    
    # 2. Try Expression Evaluator (for 'What is 1+1?' type problems)
    patterns = [r'What is\s*\$?([^?$]+)\$?\?', r'Calculate\s*\$?([^$]+)\$?']
    for pat in patterns:
        m = re.search(pat, problem, re.IGNORECASE)
        if m:
            try:
                expr = clean_latex(m.group(1))
                return int(simplify(sympify(expr))) % 100000
            except:
                pass
                
    # 3. Fallback
    return 0  # Placeholder return

# --- ⬆️ END OF CUSTOM LOGIC ⬆️ ---

# Quick sanity check
test_q = 'What is $10-3$?'
print(f"Example Predction:\nInput: {test_q}\nOutput: {solve_math_problem(test_q)}")

## 3. 🔌 The Inference Server Interface

This is the crucial part that connects your solver to Kaggle's Gateway.

**⚠️ CRITICAL DETAILS:**
1. The Gateway unpacks the DataFrame, so `predict` receives **2 separate Series** arguments (`id` and `problem`).
2. You **MUST return a DataFrame** (not a Series or int) for the Gateway to concatenate it correctly.

In [ ]:
from kaggle_evaluation.aimo_3_inference_server import AIMO3InferenceServer

def predict(id_series: pl.Series, problem_series: pl.Series) -> pl.DataFrame:
    """
    The Interface Function triggered by the Gateway.
    
    Args:
        id_series (pl.Series): A series containing the Problem ID (size 1)
        problem_series (pl.Series): A series containing the Problem Text (size 1)
        
    Returns:
        pl.DataFrame: A DataFrame with a single 'answer' column.
    """
    # 1. Unwrap the data
    problem_id = id_series[0]
    problem_text = problem_series[0]
    
    # 2. Call your custom solver
    answer = solve_math_problem(problem_text)
    
    # 3. Log (visible in submission logs)
    print(f"[{problem_id}] Problem: {problem_text[:30]}... -> Answer: {answer}")
    
    # 4. Return DataFrame with 'answer' column
    return pl.DataFrame({'answer': [answer]})

## 4. 🚀 Launch & Verify

This section handles the logic to either:
- **Serve**: When running in the actual competition (hidden test set).
- **Test Locally**: When you just run the notebook Interactive mode. It simulates the Gateway using `test.csv`.

In [ ]:
server = AIMO3InferenceServer(predict)

if IS_RERUN:
    # 🟢 SITUATION A: COMPETITION EXECUTION
    # We are being verified by the real Gateway. Start the server and wait.
    print("🚀 Starting Inference Server for Competition Rerun...")
    server.serve()
else:
    # 🟡 SITUATION B: LOCAL TESTING
    # We are running interactively. Simulate the gateway to verify everything works.
    print("🧪 Running Local Gateway Simulation...")
    
    # Define path to test data
    if IS_KAGGLE:
        test_path = '/kaggle/input/ai-mathematical-olympiad-progress-prize-3/test.csv'
    else:
        test_path = 'test.csv'
    
    # Run the simulation
    if os.path.exists(test_path):
        try:
            server.run_local_gateway(data_paths=(test_path,))
            print("\n✅ Local Gateway Simulation Completed Successfullly!")
            
            # Check the output
            if os.path.exists('submission.parquet'):
                result = pl.read_parquet('submission.parquet')
                print(f"\n📄 Generated submission.parquet (Shape: {result.shape}):")
                print(result)
            else:
                print("❌ Error: submission.parquet was NOT generated.")
        except Exception as e:
            print(f"❌ Local Gateway Error: {e}")
    else:
        print(f"⚠️ Warning: Test file not found at {test_path}. Skipping local test.")

---
### 🏆 Ready to Submit?
If you see the "✅ Local Gateway Simulation Completed" message above with a valid `submission.parquet` output, you are ready!

1. **Save Version** -> **Save & Run All (Commit)**
2. Go to Viewer -> **Submit**

*Good luck!*